<a href="https://colab.research.google.com/github/ojuba-org/arabic-ml-data/blob/master/tashkeel_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tensorflow documentation

* [v2](https://www.tensorflow.org/tutorials/text/text_generation)
* [v1](https://github.com/tensorflow/docs/blob/master/site/en/r1/tutorials/sequences/text_generation.ipynb)

In [2]:
%tensorflow_version 2.x

In [1]:
import sys
import os
import time

In [3]:
import re
import unicodedata

In [4]:
import numpy as np

In [5]:
import tensorflow as tf

In [6]:
#tf.enable_eager_execution()

In [7]:
#import tensorflow.compat.v1 as tf

In [8]:
print("tf version = {} and py version = {}".format(tf.__version__, sys.version))

tf version = 2.4.1 and py version = 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]


In [9]:
assert sys.version_info.major == 3, 'please use python 3'
assert tf.test.gpu_device_name()!='', 'no GPU, please enable GPU'

In [10]:
#from tensorflow.keras.layers.experimental import preprocessing


In [11]:
! curl -sSLO https://github.com/ojuba-org/arabic-ml-data/archive/refs/heads/master.zip

In [12]:
! unzip -q master.zip "arabic-ml-data-master/corpora/quran/*.txt"

In [13]:
! cat arabic-ml-data-master/corpora/quran/*.txt > data.txt; wc -l data.txt

6236 data.txt


In [14]:
lines=[l.strip() for l in open('data.txt', 'r') if l.strip()]
s=set()
for l in lines: s.update(l)

In [15]:
alpha=sorted(s)
for i in alpha: print(unicodedata.name(i), repr(i))


SPACE ' '
ARABIC LETTER HAMZA 'ء'
ARABIC LETTER ALEF WITH MADDA ABOVE 'آ'
ARABIC LETTER ALEF WITH HAMZA ABOVE 'أ'
ARABIC LETTER WAW WITH HAMZA ABOVE 'ؤ'
ARABIC LETTER ALEF WITH HAMZA BELOW 'إ'
ARABIC LETTER YEH WITH HAMZA ABOVE 'ئ'
ARABIC LETTER ALEF 'ا'
ARABIC LETTER BEH 'ب'
ARABIC LETTER TEH MARBUTA 'ة'
ARABIC LETTER TEH 'ت'
ARABIC LETTER THEH 'ث'
ARABIC LETTER JEEM 'ج'
ARABIC LETTER HAH 'ح'
ARABIC LETTER KHAH 'خ'
ARABIC LETTER DAL 'د'
ARABIC LETTER THAL 'ذ'
ARABIC LETTER REH 'ر'
ARABIC LETTER ZAIN 'ز'
ARABIC LETTER SEEN 'س'
ARABIC LETTER SHEEN 'ش'
ARABIC LETTER SAD 'ص'
ARABIC LETTER DAD 'ض'
ARABIC LETTER TAH 'ط'
ARABIC LETTER ZAH 'ظ'
ARABIC LETTER AIN 'ع'
ARABIC LETTER GHAIN 'غ'
ARABIC LETTER FEH 'ف'
ARABIC LETTER QAF 'ق'
ARABIC LETTER KAF 'ك'
ARABIC LETTER LAM 'ل'
ARABIC LETTER MEEM 'م'
ARABIC LETTER NOON 'ن'
ARABIC LETTER HEH 'ه'
ARABIC LETTER WAW 'و'
ARABIC LETTER ALEF MAKSURA 'ى'
ARABIC LETTER YEH 'ي'
ARABIC FATHATAN 'ً'
ARABIC DAMMATAN 'ٌ'
ARABIC KASRATAN 'ٍ'
ARABIC FATHA 'َ'
A

In [16]:
tashkeel_set = { ch for ch in alpha if unicodedata.category(ch)=='Mn' }

In [18]:
def tokenize_tashkeel(line):
  offs=[ ix for ix, ch in enumerate(line) if ch not in tashkeel_set ]
  offs.append(len(line))
  return [ line[off: offs[ix+1]] for ix, off in enumerate(offs[:-1]) ]

In [17]:
for i in sorted(tashkeel_set): print(unicodedata.name(i), repr(i))

ARABIC FATHATAN 'ً'
ARABIC DAMMATAN 'ٌ'
ARABIC KASRATAN 'ٍ'
ARABIC FATHA 'َ'
ARABIC DAMMA 'ُ'
ARABIC KASRA 'ِ'
ARABIC SHADDA 'ّ'
ARABIC SUKUN 'ْ'


In [19]:
line=lines[0]; line

'بِسْمِ اللَّهِ الرَّحْمَنِ الرَّحِيمِ'

In [20]:
print(len(line))
tokens=tokenize_tashkeel(line)
print(tokens)
[ len(token) for token in tokens]

37
['بِ', 'سْ', 'مِ', ' ', 'ا', 'ل', 'لَّ', 'هِ', ' ', 'ا', 'ل', 'رَّ', 'حْ', 'مَ', 'نِ', ' ', 'ا', 'ل', 'رَّ', 'حِ', 'ي', 'مِ']


[2, 2, 2, 1, 1, 1, 3, 2, 1, 1, 1, 3, 2, 2, 2, 1, 1, 1, 3, 2, 1, 2]

In [28]:
(
  VOC_START,
  VOC_UNKNOWN,
  VOC_UNKNOWN_TASHKEEL, VOC_UNKNOWN_SHADDA,
  VOC_UNKNOWN_LETTER, VOC_UNKNOWN_HAMZA, VOC_UNKNOWN_TEH_OR_HAA, VOC_UNKNOWN_MAQSORA,
  VOC_FATHATAN, VOC_DAMMATAN, VOC_KASRATAN, 
  VOC_FATHA, VOC_DAMMA, VOC_KASRA,
  VOC_SHADDA_FATHA, VOC_SHADDA_DAMMA, VOC_SHADDA_KASRA,
  VOC_SUKUN,
  VOC_STOP,
) = range(19)

In [34]:
VOC_STOP

18

In [48]:
chars = [ ch for ch in alpha if unicodedata.category(ch)!='Mn']

In [49]:
ch2id = { ch: ix+VOC_STOP+1 for ix, ch in enumerate(chars)}

In [50]:
ch2id.update({
  "": VOC_START,
  "\n": VOC_STOP,
  "\u064B": VOC_FATHATAN, "\u064C": VOC_DAMMATAN, "\u064D": VOC_KASRATAN,
  "\u064E": VOC_FATHA, "\u064F": VOC_DAMMA, "\u0650": VOC_KASRA,
  "\u0651\u064E": VOC_SHADDA_FATHA, "\u0651\u064F": VOC_SHADDA_DAMMA, "\u0651\u0650": VOC_SHADDA_KASRA,
  "\u0651": VOC_SUKUN,
  "\u061f": VOC_UNKNOWN,
  "\u0640\u061f\u0640": VOC_UNKNOWN_TASHKEEL,
  "\u0651\u061f": VOC_UNKNOWN_SHADDA,
})

In [51]:
id2ch = { v: k for k, v in ch2id.items() }

In [55]:
voc_size = max(ch2id.values())
print(voc_size)

55


In [56]:
id2ch.update({ ix:'\u061f' for ix in range(voc_size) if ix not in id2ch})

In [57]:
id2ch

{0: '',
 1: '؟',
 2: 'ـ؟ـ',
 3: 'ّ؟',
 4: '؟',
 5: '؟',
 6: '؟',
 7: '؟',
 8: 'ً',
 9: 'ٌ',
 10: 'ٍ',
 11: 'َ',
 12: 'ُ',
 13: 'ِ',
 14: 'َّ',
 15: 'ُّ',
 16: 'ِّ',
 17: 'ّ',
 18: '\n',
 19: ' ',
 20: 'ء',
 21: 'آ',
 22: 'أ',
 23: 'ؤ',
 24: 'إ',
 25: 'ئ',
 26: 'ا',
 27: 'ب',
 28: 'ة',
 29: 'ت',
 30: 'ث',
 31: 'ج',
 32: 'ح',
 33: 'خ',
 34: 'د',
 35: 'ذ',
 36: 'ر',
 37: 'ز',
 38: 'س',
 39: 'ش',
 40: 'ص',
 41: 'ض',
 42: 'ط',
 43: 'ظ',
 44: 'ع',
 45: 'غ',
 46: 'ف',
 47: 'ق',
 48: 'ك',
 49: 'ل',
 50: 'م',
 51: 'ن',
 52: 'ه',
 53: 'و',
 54: 'ى',
 55: 'ي'}

In [23]:
import functools

In [24]:
rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    rnn(rnn_units,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model
